In [1]:
from base_types import IdxValue, DataElements
from data import Data
from plot import PricePlot
from utils import milliseconds_to_date
from base_types import DataType
import json

class SavedInfo():
    def __init__(self, data: Data, buy_points: IdxValue, sell_points: IdxValue, 
                 tops: IdxValue, bottoms: IdxValue, earn_points: IdxValue):
        self.data: Data = data
        self.buy_points: IdxValue = buy_points    # time, value
        self.sell_points: IdxValue = sell_points  # time, value
        self.tops: IdxValue = tops                # time, value
        self.bottoms: IdxValue = bottoms          # time, value
        self.earn_points: IdxValue = earn_points
    
    def get_all(self):
        return self.data, self.buy_points, self.sell_points, self.tops, self.bottoms, self.earn_points

def read_data(symbol, exp_name, start, end):
    start_str = milliseconds_to_date(start) + ' UTC+8'
    end_str = milliseconds_to_date(end + 1) + ' UTC+8'

    data = Data(symbol, DataType.INTERVAL_1MINUTE, start_str=start_str, end_str=end_str, is_futures=True)
    # print(data.start_time())
    # print(data.end_time())

    base_path = '.\\log\\{}\\{}_start_{}_end_{}_'.format(exp_name, symbol, data.start_time(), data.end_time())
    trade_info_path = base_path + 'trade_info.json'
    vertices_path = base_path + 'vertices.json'
    earn_path = base_path + 'earn_points.json'

    with open(trade_info_path, 'r') as f:
        json_data = f.read()
        trade_info = json.loads(json_data)

    with open(vertices_path, 'r') as f:
        json_data = f.read()
        vertices = json.loads(json_data)

    with open(earn_path, 'r') as f:
        json_data = f.read()
        earn_points_dict = json.loads(json_data)

    # print(trade_info.keys())
    # print(vertices.keys())
    # print(earn_points_dict.keys())

    buy_points = IdxValue(trade_info['buy_time'], trade_info['buy_price'])
    sell_points = IdxValue(trade_info['sell_time'], trade_info['sell_price'])
    tops = IdxValue(vertices['top_time'], vertices['top_value'])
    bottoms = IdxValue(vertices['bottom_time'], vertices['bottom_value'])
    earn_points = IdxValue(earn_points_dict['earn_idx'], earn_points_dict['earn_value'])
    
    return SavedInfo(data, buy_points, sell_points, tops, bottoms, earn_points)
    


In [2]:
import matplotlib.pyplot as plt
import mpl_finance
from matplotlib import ticker
from base_types import DataElements
import numpy as np

def get_data_from_time(time_value: IdxValue, begin, end):
    timestamp = np.array(time_value.idx)
    value = np.array(time_value.value)
    idx = (timestamp >= begin) & (timestamp <= end)

    return timestamp[idx], value[idx]


def get_plot_data(saved_info: SavedInfo, start_idx=None, end_idx=None):
    data, buy_points, sell_points, tops, bottoms, earn_points = saved_info.get_all()
    subdata = data.data.loc[start_idx:end_idx, [DataElements.OPEN.value, DataElements.HIGH.value, 
                                DataElements.LOW.value, DataElements.CLOSE.value, DataElements.OPEN_TIME.value]].copy().reset_index()
    subdata['timestamp'] = subdata[DataElements.OPEN_TIME.value].map(milliseconds_to_date)


    start_time = subdata[DataElements.OPEN_TIME.value].values[0]
    end_time = subdata[DataElements.OPEN_TIME.value].values[-1]

    points = [ 
        PricePlot.Points(*get_data_from_time(buy_points, start_time, end_time), s=90, c='r', label='buy'),
        PricePlot.Points(*get_data_from_time(sell_points, start_time, end_time), s=90, c='g', label='sell'),
        PricePlot.Points(*get_data_from_time(tops, start_time, end_time), s=30, c='b', label='top'),
        PricePlot.Points(*get_data_from_time(bottoms, start_time, end_time), s=30, c='y', label='bottoms'),
    ]

    for point in points:
        data.time_list_to_idx(point.idx)
    
    return {'subdata': subdata, 'points': points, 'earn_points': earn_points}

# subdata_multi_0_5, points_multi_0_5 = get_plot_data(info_multi_0_5, None, None)
# plot_data_new_policy = get_plot_data(info_new_policy)
# plot_data = get_plot_data(info)


In [12]:
def plot_with_data_points(all_data_points, plot_candle: bool):
    figure_num = len(all_data_points)
    subplot = None
    for num in range(figure_num):
        data = all_data_points[num]['subdata']
        points = all_data_points[num]['points']

        subplot = plt.subplot2grid((figure_num+1, 1), (num, 0), rowspan=1, colspan=1, sharex=subplot, sharey=subplot)

        if plot_candle:
            mpl_finance.candlestick_ohlc(ax=subplot, quotes=data.values,
                                        width=0.7, colorup='g', colordown='r')
        else:
            subplot.plot(range(0, len(data)), data[DataElements.CLOSE.value].values,  # type: ignore
                        color="gray", linewidth=1.0, label='base')

        subplot.set_ylabel('Price') # type: ignore

        if points:
            for p in points:
                subplot.scatter(p.idx, p.value, s=p.s, c=p.c, label=p.label)  # type: ignore

    subplot = plt.subplot2grid((figure_num+1, 1), (figure_num, 0), rowspan=1, colspan=1, sharex=subplot)
    colors = ['b', 'r', 'g', 'y']
    for num in range(figure_num):
        earn_points = all_data_points[num]['earn_points']
        subplot.plot(earn_points.idx + [len(earn_points.idx)-1], earn_points.value + [earn_points.value[-1]], # type: ignore
                     color=colors[num], linewidth=1.0, label=str(num))

    def format_date(x, pos):
        if x < 0 or x > len(data) - 1:
            return ''
        return data['timestamp'].values[int(x)]
    subplot.xaxis.set_major_formatter(ticker.FuncFormatter(format_date)) # type: ignore

%matplotlib qt5
all_data = [
    # get_plot_data(read_data(
    #         symbol = 'LUNA2BUSD',
    #         exp_name = 'ksol_0.9_0_0.6_mfp',
    #         start = 1654052700000,
    #         end = 1656658799999
    # )),
    get_plot_data(read_data(
            symbol = 'LUNA2BUSD',
            exp_name = 'ksol_0.9_0.15_0.6_mfp_frontEn',
            start = 1654052700000,
            end = 1656658799999
    )),
    get_plot_data(read_data(
            symbol = 'LUNA2BUSD',
            exp_name = 'ksol_0.9_0.2_0.6_mfp_frontEn',
            start = 1654052700000,
            end = 1656658799999
    )),
    
]
plot_with_data_points(all_data, False)